# This file is used to train and test the model.

In [1]:
import torch
import numpy as np
import pandas as pd
from model import SC_LSTM as Model

# Device init

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    !nvidia-smi
    print(torch.cuda.get_device_name(0))

else:
    print("No GPU :(")

Sun Jun 25 03:02:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti      Off| 00000000:01:00.0  On |                  N/A |
| N/A   41C    P0               17W /  N/A|    721MiB /  4096MiB |     34%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
device = 'cpu'

## Init the model

In [4]:
# Defining a model
in_features = 9
out_features = 3
n_layers = 1
hidden_size = 10

SC_LSTM = Model(in_features, n_layers, hidden_size, out_features).to(device)

## Loading the data

In [5]:
from Data.Preprocessor import Preprocessor

preproc = Preprocessor()

data = pd.read_csv('../Data/schedule_v3.csv')
data

,Label Number,Duration,Importance,Start Time,Date
0,3,75,1,6:30,22/06/2023
1,5,90,2,8:15,22/06/2023
2,0,30,0,10:00,22/06/2023
3,1,30,1,10:45,22/06/2023
4,6,15,0,11:30,22/06/2023
...,...,...,...,...,...
967,4,90,2,18:15,01/09/2023
968,1,30,1,19:30,01/09/2023
969,6,45,0,20:45,01/09/2023
970,3,45,1,22:00,01/09/2023


In [6]:
data = preproc.preprocess(data)
data

,Label Number,Duration,Importance,Time_Min_sin,Time_Min_cos,Date_Day_sin,Date_Day_cos,Date_Month_sin,Date_Month_cos
0,3,0.571429,1,0.988948,-0.148264,-0.968077,-0.250653,-8.660254e-01,-0.5
1,5,0.714286,2,0.818625,-0.574329,-0.968077,-0.250653,-8.660254e-01,-0.5
2,0,0.142857,0,0.475947,-0.879474,-0.968077,-0.250653,-8.660254e-01,-0.5
3,1,0.142857,1,0.293250,-0.956036,-0.968077,-0.250653,-8.660254e-01,-0.5
4,6,0.000000,0,0.099046,-0.995083,-0.968077,-0.250653,-8.660254e-01,-0.5
...,...,...,...,...,...,...,...,...,...
967,4,0.714286,2,-0.993309,0.115485,0.201299,0.979530,-2.449294e-16,1.0
968,1,0.142857,1,-0.901991,0.431754,0.201299,0.979530,-2.449294e-16,1.0
969,6,0.285714,0,-0.712928,0.701237,0.201299,0.979530,-2.449294e-16,1.0
970,3,0.285714,1,-0.446609,0.894729,0.201299,0.979530,-2.449294e-16,1.0


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Testing the model

In [8]:
# Nevermind of this shitty code :)
SC_LSTM.eval()

with torch.no_grad():
    for i in range(0, len(data)):
        x = torch.tensor(data[i]).float().to(device)
        x = x.view(1, 1, -1)
        print(f"X shape{x.shape})")
        y = SC_LSTM(x, None, None)
        # print(y)

X shapetorch.Size([1, 1, 9]))
tensor([[[-0.0431, -0.1241,  0.0134,  0.0935,  0.1168,  0.0040,  0.2212,
          -0.1237, -0.0321, -0.1089]]]) tensor([[[-0.0431, -0.1241,  0.0134,  0.0935,  0.1168,  0.0040,  0.2212,
          -0.1237, -0.0321, -0.1089]]]) tensor([[[-0.0995, -0.2977,  0.0192,  0.1755,  0.2106,  0.0077,  0.3693,
          -0.3087, -0.0577, -0.1554]]])
X shapetorch.Size([1, 1, 9]))
tensor([[[-0.0249, -0.1112, -0.0048,  0.0792,  0.1338, -0.0253,  0.2128,
          -0.1554, -0.0098, -0.1030]]]) tensor([[[-0.0249, -0.1112, -0.0048,  0.0792,  0.1338, -0.0253,  0.2128,
          -0.1554, -0.0098, -0.1030]]]) tensor([[[-0.0565, -0.2883, -0.0070,  0.1371,  0.2407, -0.0495,  0.3476,
          -0.3714, -0.0162, -0.1482]]])
X shapetorch.Size([1, 1, 9]))
tensor([[[-0.0644, -0.1334,  0.0123,  0.0716,  0.1285,  0.0151,  0.1804,
          -0.0803, -0.0707, -0.1110]]]) tensor([[[-0.0644, -0.1334,  0.0123,  0.0716,  0.1285,  0.0151,  0.1804,
          -0.0803, -0.0707, -0.1110]]]) tensor